# Preprocessing i klasyfikacja wieloklasowa

Dla tokenizacji i normalizacji tekstu wykorzystamy pakiet `spacy`.

In [ ]:
import helper as h

import pandas as pd
import numpy as np

import re
import string

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import spacy
from html import unescape


import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
nlp = spacy.load("en_core_web_sm")

#### <b>Wczytajmy dane ;-)</b>

In [ ]:
df = pd.read_hdf("./tweets_airline.h5")

print(df.shape)
df.sample(5)

Sprawdźmy, jakich firm to dotyczy.

In [ ]:
df['airline'].value_counts()

Zmienna docelowa:

In [ ]:
df['airline_sentiment'].value_counts()

Jak widać, mamy ponad 9 tys. negatywnych opinii i tylko 2363 pozytywnych. Co jest dość naturalne ... bo zwykle niezadowolony wkurzony klient opowie o 10 osobom, a zadowolony tylko jednej. Ta proporcja jest tu zauważalna.

Zamiast konwertować opinię do dwu klas (`negative` i `positive`) pozostaniemy przy 3, a <b>3 implikuje chaos ...</b> :-)

In [ ]:
df['airline_sentiment_target'], labels = df['airline_sentiment'].factorize()

## Base Line

Zacznijmy jak "najszybciej" od uzyskania wyników, aby móc porównywać, na ile nasze zmiany są pomocne. Dlatego nie róbmy żadnego "preprocessing" i użyjmy najprostszego `vectorizer`, czyli `CountVectorizer`.

Trenujemy model i sprawdzamy wynik.

In [ ]:
X = df['text'].values
y = df['airline_sentiment_target'].values

In [ ]:
vectorizer_kwargs={'max_features':20}
kwargs = {'scoring': 'f1_micro'}

h.use_vectorizer_and_run_models(X, y, CountVectorizer, vectorizer_kwargs, kwargs)

W tym przypadku bierzemy pod uwagę 20 tokenów `{'max_features':20}`.

```
['americanair', 'and', 'flight', 'for', 'in', 'is', 'it', 'jetblue', 'me', 'my', 'of', 'on', 'southwestair', 'that', 'the', 'to', 'united', 'usairways', 'you', 'your']
```

Cześć z nich to nazwa firm lotniczych: `americanair`, `jetblue`, `southwestair`, `usairways`, `united`. Brakuje jeszcze jednej `Delta` oraz `Virgin America`. Swoją drogą `jetblue` trafiło tu przy okazji ;).

Są też ogólne słowa, takie jak: `and`, `for`, `in`, `is`, `it`. Owszem te słowa są często używane, ale czy wnoszą znaczącą informację? Zwiększmy liczbę tokenów do 100 (`'max_features':100`).


In [ ]:
vectorizer_kwargs={'max_features': 100}
kwargs = {'scoring': 'f1_micro'}

h.use_vectorizer_and_run_models(X, y, CountVectorizer, vectorizer_kwargs, kwargs)

Wynik nam znacząco sie poprawił. Mamy już `f1_micro=74%`. Tym razem zwyciężył `svm`. Spróbujmy teraz użyć `tf-idf`.

Te tokeny, zostały użyte:
```
['about', 'after', 'again', 'airline', 'all', 'am', 'americanair', 'amp', 'an', 'and', 'any', 'are', 'as', 'at', 'back', 'bag', 'be', 'been', 'but', 'by', 'call', 'can', 'cancelled', 'co', 'customer', 'delayed', 'do', 'don', 'flight', 'flightled', 'flights', 'for', 'from', 'gate', 'get', 'got', 'had', 'has', 'have', 'help', 'hold', 'hour', 'hours', 'how', 'http', 'if', 'in', 'is', 'it', 'jetblue', 'just', 'late', 'like', 'me', 'more', 'my', 'need', 'no', 'not', 'now', 'of', 'on', 'one', 'or', 'our', 'out', 'over', 'phone', 'plane', 'please', 'service', 'so', 'southwestair', 'still', 'thank', 'thanks', 'that', 'the', 'there', 'they', 'this', 'time', 'to', 'today', 'united', 'up', 'us', 'usairways', 've', 'virginamerica', 'was', 'we', 'what', 'when', 'why', 'will', 'with', 'would', 'you', 'your']
```

Widać np. token `cancelled` (prawdopodobnie występuje głównie w negatywnym wydźwięku).


## Zagadka na dziś [tj. 4 listopada 2025 r.]

Czy `TfidfVectorizer` poradzi sobie lepiej?

_Też warto sprawdzić dwa przypadki np.:`{'max_features':20}` oraz `{'max_features':100}`, aby sprawdzić, czy uzyslujemy poprawę._

In [ ]:
vectorizer_kwargs={'max_features':20}
kwargs = {'scoring': 'f1_micro'}

# h.use_vectorizer_and_run_models(X, y, TfidfVectorizer, vectorizer_kwargs, kwargs)